# Master-Detail

> Responsive sidebar navigation pattern with master list and detail content area. On mobile devices, the sidebar is hidden in a drawer that can be toggled. On desktop (lg+ screens), the sidebar is always visible.

In [ ]:
#| default_exp patterns.master_detail

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional, Callable, List, Union
from dataclasses import dataclass, field
from fasthtml.common import *
from fasthtml.svg import Svg, Path
from fastcore.basics import patch

from cjm_fasthtml_interactions.core.context import InteractionContext
from cjm_fasthtml_interactions.core.html_ids import InteractionHtmlIds
from cjm_fasthtml_daisyui.components.navigation.menu import menu, menu_title, menu_modifiers
from cjm_fasthtml_daisyui.components.layout.divider import divider
from cjm_fasthtml_daisyui.components.layout.drawer import (
    drawer, drawer_toggle, drawer_content, drawer_side, drawer_overlay, drawer_modifiers
)
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_styles
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_sizes
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui
from cjm_fasthtml_daisyui.utilities.border_radius import border_radius
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_direction, flex, shrink, items, justify, gap
)
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w, h, min_h, size_util
from cjm_fasthtml_tailwind.utilities.typography import font_weight
from cjm_fasthtml_tailwind.utilities.layout import position, overflow, display_tw
from cjm_fasthtml_tailwind.utilities.transitions_and_animation import transition, duration
from cjm_fasthtml_tailwind.core.base import combine_classes

## Detail Item Definition

The `DetailItem` class defines a single item in the master list. Each item has:
- Unique identifier
- Display label for the master list
- Render function that generates the detail view UI
- Optional badge text and color
- Optional icon
- Optional data loader for fetching required data

In [ ]:
#| export
@dataclass
class DetailItem:
    """Definition of a single item in the master-detail pattern."""
    
    id: str  # Unique identifier
    label: str  # Display text in master list
    render: Callable[[InteractionContext], Any]  # Function to render detail view
    badge_text: Optional[str] = None  # Optional badge text (e.g., "configured", "3 items")
    badge_color: Optional[str] = None  # Badge color class (e.g., badge_colors.success)
    icon: Optional[Any] = None  # Optional icon element
    data_loader: Optional[Callable[[Any], Dict[str, Any]]] = None  # Data loading function
    load_on_demand: bool = True  # Whether to load content only when item is selected

## Detail Item Group Definition

The `DetailItemGroup` class groups related items in a collapsible section. Each group has:
- Unique identifier
- Display title
- List of items in the group
- Collapsible state control
- Optional icon

In [ ]:
#| export
@dataclass
class DetailItemGroup:
    """Group of related detail items in a collapsible section."""
    
    id: str  # Group identifier
    title: str  # Group display title
    items: List[DetailItem]  # Items in this group
    default_open: bool = True  # Whether group is expanded by default
    icon: Optional[Any] = None  # Optional group icon
    badge_text: Optional[str] = None  # Optional badge for the group
    badge_color: Optional[str] = None  # Badge color for the group

## MasterDetail Class

The `MasterDetail` class manages a responsive master-detail interface. It:
- Renders master list (sidebar) with items and optional groups
- Manages active item selection
- Renders detail content for selected item
- Supports badges and status indicators
- Supports hierarchical grouping with collapsible sections
- **Responsive design**: Drawer toggleable on mobile, always visible on desktop (lg+)
- **Mobile-friendly**: Includes hamburger menu button for navigation
- Generates routes automatically
- Provides OOB updates for coordinated UI changes

In [ ]:
#| export
class MasterDetail:
    """Manage master-detail interfaces with sidebar navigation and detail content area."""
    
    def __init__(
        self,
        interface_id: str,  # Unique identifier for this interface
        items: List[Union[DetailItem, DetailItemGroup]],  # List of items/groups
        default_item: Optional[str] = None,  # Default item ID (defaults to first item)
        container_id: str = InteractionHtmlIds.MASTER_DETAIL_CONTAINER,  # HTML ID for container
        master_id: str = InteractionHtmlIds.MASTER_DETAIL_MASTER,  # HTML ID for master list
        detail_id: str = InteractionHtmlIds.MASTER_DETAIL_DETAIL,  # HTML ID for detail area
        master_width: str = "w-64",  # Tailwind width class for master list
        master_title: Optional[str] = None,  # Optional title for master list
        show_on_htmx_only: bool = False  # Whether to show full interface for non-HTMX requests
    ):
        """Initialize master-detail manager."""
        self.interface_id = interface_id
        self.items = items
        self.container_id = container_id
        self.master_id = master_id
        self.detail_id = detail_id
        self.master_width = master_width
        self.master_title = master_title
        self.show_on_htmx_only = show_on_htmx_only
        
        # Build item index for quick lookup (flatten groups)
        self.item_index = {}
        for item in items:
            if isinstance(item, DetailItemGroup):
                for sub_item in item.items:
                    self.item_index[sub_item.id] = sub_item
            else:
                self.item_index[item.id] = item
        
        # Set default item
        if default_item and default_item in self.item_index:
            self.default_item = default_item
        elif self.item_index:
            self.default_item = list(self.item_index.keys())[0]
        else:
            self.default_item = None

## Item Management Methods

In [ ]:
#| export
@patch
def get_item(self:MasterDetail, 
             item_id: str  # Item identifier
            ) -> Optional[DetailItem]:  # DetailItem or None
    """Get item by ID."""
    return self.item_index.get(item_id)

## Context and Rendering Methods

In [ ]:
#| export
@patch
def create_context(self:MasterDetail, 
                   request: Any,  # FastHTML request object
                   sess: Any,  # FastHTML session object
                   item: DetailItem  # Current item
                  ) -> InteractionContext:  # Interaction context for rendering
    """Create interaction context for an item."""
    # Load data if item has data loader
    data = {}
    if item.data_loader:
        data = item.data_loader(request)
    
    return InteractionContext(
        state={},  # Master-detail typically doesn't maintain state between selections
        request=request,
        session=sess,
        data=data
    )

In [ ]:
#| export
@patch
def render_master(self:MasterDetail,
                  active_item_id: str,  # Currently active item ID
                  item_route_func: Callable[[str], str],  # Function to generate item route
                  include_wrapper: bool = True  # Whether to include outer wrapper div
                 ) -> FT:  # Master list element
    """Render master list (sidebar) with items and groups."""
    menu_ul = Ul(
        *self._render_menu_items(active_item_id, item_route_func),
        id=self.master_id,
        cls=combine_classes(
            menu,
            bg_dui.base_200,
            self.master_width,
            p(4),
            h.auto,
            border_radius.box
        )
    )
    
    if include_wrapper:
        return Div(
            menu_ul,
            cls=combine_classes(
                shrink(0),
                position.sticky,
                overflow.y.auto,
                h.full
            )
        )
    else:
        return menu_ul

In [ ]:
#| export
@patch
def _render_menu_items(self:MasterDetail,
                       active_item_id: str,  # Currently active item ID
                       item_route_func: Callable[[str], str]  # Function to generate item route
                      ) -> List[FT]:  # List of menu item elements
    """Render menu items and groups (internal helper)."""
    menu_items = []
    
    # Add menu title if provided
    if self.master_title:
        menu_items.append(
            Li(
                Span(self.master_title, cls=str(menu_title))
            )
        )
    
    # Process items and groups
    for entry in self.items:
        if isinstance(entry, DetailItemGroup):
            # Handle group with collapsible section
            group = entry
            
            # Create submenu items for each item in the group
            submenu_items = []
            for item in group.items:
                is_active = active_item_id == item.id
                
                # Build item content with optional icon and badge
                item_content = []
                if item.icon:
                    item_content.append(item.icon)
                
                item_content.append(
                    Span(
                        item.label,
                        cls=str(font_weight.medium if is_active else "")
                    )
                )
                
                if item.badge_text:
                    badge_cls = combine_classes(
                        badge,
                        item.badge_color or badge_colors.success,
                        badge_sizes.xs,
                        m.l(2)
                    )
                    item_content.append(
                        Span(item.badge_text, cls=badge_cls)
                    )
                
                submenu_items.append(
                    Li(
                        A(
                            Div(
                                *item_content,
                                cls=combine_classes(flex_display, items.center, justify.between, w.full, gap(2))
                            ),
                            href=item_route_func(item.id),
                            hx_get=item_route_func(item.id),
                            hx_target=InteractionHtmlIds.as_selector(self.detail_id),
                            hx_swap="innerHTML",
                            hx_push_url="true",
                            cls=combine_classes(
                                menu_modifiers.active if is_active else "",
                                transition.colors,
                                duration(200)
                            )
                        ),
                        id=InteractionHtmlIds.master_item(item.id)
                    )
                )
            
            # Build group header with optional badge
            group_header_content = []
            if group.icon:
                group_header_content.append(group.icon)
            
            group_header_content.append(
                Span(group.title, cls=str(font_weight.medium))
            )
            
            if group.badge_text:
                badge_cls = combine_classes(
                    badge,
                    group.badge_color or badge_colors.success,
                    badge_sizes.xs,
                    m.l(2)
                )
                group_header_content.append(
                    Span(group.badge_text, cls=badge_cls)
                )
            
            # Check if any item in group is active (to auto-expand)
            group_has_active = any(item.id == active_item_id for item in group.items)
            
            # Create the collapsible group item
            menu_items.append(
                Li(
                    Details(
                        Summary(
                            Div(
                                *group_header_content,
                                cls=combine_classes(flex_display, items.center, justify.between, w.full, gap(2))
                            )
                        ),
                        Ul(*submenu_items),
                        open=group.default_open or group_has_active
                    ),
                    id=InteractionHtmlIds.master_group(group.id)
                )
            )
        else:
            # Handle individual item
            item = entry
            is_active = active_item_id == item.id
            
            # Build item content with optional icon and badge
            item_content = []
            if item.icon:
                item_content.append(item.icon)
            
            item_content.append(
                Span(
                    item.label,
                    cls=str(font_weight.medium if is_active else "")
                )
            )
            
            if item.badge_text:
                badge_cls = combine_classes(
                    badge,
                    item.badge_color or badge_colors.success,
                    badge_sizes.xs,
                    m.l(2)
                )
                item_content.append(
                    Span(item.badge_text, cls=badge_cls)
                )
            
            menu_items.append(
                Li(
                    A(
                        Div(
                            *item_content,
                            cls=combine_classes(flex_display, items.center, justify.between, w.full, gap(2))
                        ),
                        href=item_route_func(item.id),
                        hx_get=item_route_func(item.id),
                        hx_target=InteractionHtmlIds.as_selector(self.detail_id),
                        hx_swap="innerHTML",
                        hx_push_url="true",
                        cls=combine_classes(
                            menu_modifiers.active if is_active else "",
                            transition.colors,
                            duration(200)
                        )
                    ),
                    id=InteractionHtmlIds.master_item(item.id)
                )
            )
    
    return menu_items

In [ ]:
#| export
@patch
def render_master_oob(self:MasterDetail,
                      active_item_id: str,  # Currently active item ID
                      item_route_func: Callable[[str], str]  # Function to generate item route
                     ) -> FT:  # Master list with OOB swap attribute
    """Render master list with OOB swap attribute for coordinated updates."""
    master = self.render_master(
        active_item_id=active_item_id,
        item_route_func=item_route_func,
        include_wrapper=False
    )
    master.attrs['hx-swap-oob'] = 'true'
    return master

In [ ]:
#| export
@patch
def render_detail(self:MasterDetail,
                  item: DetailItem,  # Item to render
                  ctx: InteractionContext  # Interaction context
                 ) -> FT:  # Detail content
    """Render detail content for an item."""
    return item.render(ctx)

In [ ]:
#| export
@patch
def render_full_interface(self:MasterDetail,
                         active_item_id: str,  # Currently active item ID
                         item_route_func: Callable[[str], str],  # Function to generate item route
                         request: Any,  # FastHTML request object
                         sess: Any  # FastHTML session object
                        ) -> FT:  # Complete master-detail interface
    """Render complete responsive master-detail interface with drawer for mobile."""
    active_item = self.get_item(active_item_id)
    if not active_item:
        active_item = self.get_item(self.default_item)
        active_item_id = self.default_item
    
    # Generate unique drawer ID for this interface instance
    drawer_id = f"master-detail-drawer-{self.interface_id}"
    
    # Create menu icon SVG for mobile toggle button
    menu_icon = Svg(
        Path(
            stroke_linecap="round",
            stroke_linejoin="round",
            d="M3.75 6.75h16.5M3.75 12h16.5m-16.5 5.25h16.5"
        ),
        xmlns="http://www.w3.org/2000/svg",
        fill="none",
        viewBox="0 0 24 24",
        stroke_width="1.5",
        stroke="currentColor",
        cls=str(size_util("1.5em"))
    )
    
    # Render master list menu without wrapper
    master_menu = Ul(
        *self._render_menu_items(active_item_id, item_route_func),
        id=self.master_id,
        cls=combine_classes(
            menu,
            bg_dui.base_200,
            text_dui.base_content,
            self.master_width,
            p(4),
            min_h.full
        )
    )
    
    # Create detail content based on load_on_demand setting
    if active_item.load_on_demand:
        detail_content = Div(
            hx_get=item_route_func(active_item_id),
            hx_trigger="load",
            hx_swap="innerHTML",
            id=self.detail_id,
            cls=str(p(6))
        )
    else:
        detail_content = Div(
            self.render_detail(active_item, self.create_context(request, sess, active_item)),
            id=self.detail_id,
            cls=str(p(6))
        )
    
    # Create responsive drawer layout
    # Mobile: drawer hidden by default, toggle button visible
    # Desktop (lg+): drawer always open, toggle button hidden
    return Div(
        # Hidden checkbox that controls drawer state
        Input(
            id=drawer_id,
            type="checkbox",
            cls=str(drawer_toggle)
        ),
        
        # Drawer content area (main content on desktop, with toggle button)
        Div(
            # Mobile menu toggle button (hidden on lg+ screens)
            Label(
                menu_icon,
                _for=drawer_id,
                aria_label="Open navigation menu",
                cls=combine_classes(
                    btn,
                    btn_styles.ghost,
                    display_tw.hidden.lg,
                    m.b(4),
                    m.l(4),
                    m.t(4)
                )
            ),
            
            # Detail content area
            detail_content,
            
            cls=str(drawer_content)
        ),
        
        # Drawer side (master list navigation)
        Div(
            # Overlay that closes drawer when clicked on mobile
            Label(
                _for=drawer_id,
                aria_label="Close sidebar",
                cls=str(drawer_overlay)
            ),
            
            # Master list menu
            master_menu,
            
            cls=str(drawer_side)
        ),
        
        id=self.container_id,
        cls=combine_classes(
            drawer,
            drawer_modifiers.open.lg  # Always open on lg+ screens
        )
    )

## Route Generation

In [ ]:
#| export
@patch
def create_router(self:MasterDetail,
                  prefix: str = ""  # URL prefix for routes (e.g., "/media")
                 ) -> APIRouter:  # APIRouter with generated routes
    """Create FastHTML router with generated routes for this master-detail interface."""
    router = APIRouter(prefix=prefix)

    # Store reference to interface in router for access in route handlers
    router.master_detail = self

    # Index route - show full interface
    @router
    def index(request, sess, item_id: str = None):
        """Main master-detail interface entry point."""
        from cjm_fasthtml_app_core.core.htmx import is_htmx_request
        
        # Determine current item
        current_item_id = item_id or self.default_item
        
        # For HTMX requests, return just the detail content if configured to do so
        if self.show_on_htmx_only and is_htmx_request(request):
            item = self.get_item(current_item_id)
            if item:
                ctx = self.create_context(request, sess, item)
                return self.render_detail(item, ctx)
        
        # For non-HTMX or when full interface is needed
        return self.render_full_interface(
            active_item_id=current_item_id,
            item_route_func=lambda iid: detail.to(item_id=iid),
            request=request,
            sess=sess
        )

    # Detail route - load individual item detail content
    @router
    def detail(request, sess, item_id: str):
        """Load detail content for a specific item."""
        from cjm_fasthtml_app_core.core.htmx import is_htmx_request
        
        item = self.get_item(item_id)
        if not item:
            # Invalid item, redirect to default
            item_id = self.default_item
            item = self.get_item(item_id)
        
        # For HTMX requests, return just the detail content
        if is_htmx_request(request):
            ctx = self.create_context(request, sess, item)
            return self.render_detail(item, ctx)
        
        # For direct navigation (refresh or direct URL access), return full interface
        return self.render_full_interface(
            active_item_id=item_id,
            item_route_func=lambda iid: detail.to(item_id=iid),
            request=request,
            sess=sess
        )

    return router

## Usage Example

Here's a complete example showing how to create a master-detail interface:

**Note:** The interface is automatically responsive:
- **Mobile (< lg):** Master list hidden in a drawer with hamburger menu toggle
- **Desktop (≥ lg):** Master list always visible as a sidebar

In [ ]:
# Example: Simple file browser with groups
from fasthtml.common import Div, H2, P, H3, Ul, Li

# Define render functions for each item
def render_file_overview(ctx: InteractionContext):
    """Render file details."""
    file_data = ctx.get_data("file", {})
    return Div(
        H2(f"File: {file_data.get('name', 'Unknown')}"),
        P(f"Size: {file_data.get('size', 0)} bytes"),
        P(f"Modified: {file_data.get('modified', 'N/A')}")
    )

def render_folder_overview(ctx: InteractionContext):
    """Render folder contents."""
    folder_data = ctx.get_data("folder", {})
    return Div(
        H2(f"Folder: {folder_data.get('name', 'Unknown')}"),
        P(f"Items: {folder_data.get('item_count', 0)}"),
        H3("Contents:"),
        Ul(*[Li(item) for item in folder_data.get('items', [])])
    )

# Optional data loaders
def load_file_data(request):
    """Load file metadata."""
    return {
        "file": {
            "name": "document.txt",
            "size": 1024,
            "modified": "2025-01-15"
        }
    }

def load_folder_data(request):
    """Load folder contents."""
    return {
        "folder": {
            "name": "Documents",
            "item_count": 3,
            "items": ["file1.txt", "file2.pdf", "file3.doc"]
        }
    }

# Create the master-detail interface with groups
file_browser = MasterDetail(
    interface_id="file_browser",
    master_title="File Browser",
    items=[
        DetailItemGroup(
            id="documents",
            title="Documents",
            items=[
                DetailItem(
                    id="doc1",
                    label="document.txt",
                    render=render_file_overview,
                    data_loader=load_file_data,
                    badge_text="1KB",
                    badge_color=badge_colors.info
                ),
                DetailItem(
                    id="doc2",
                    label="report.pdf",
                    render=render_file_overview,
                    badge_text="2KB",
                    badge_color=badge_colors.info
                )
            ],
            badge_text="2 files"
        ),
        DetailItemGroup(
            id="media",
            title="Media",
            items=[
                DetailItem(
                    id="img1",
                    label="photo.jpg",
                    render=render_file_overview,
                    badge_text="512KB",
                    badge_color=badge_colors.warning
                )
            ],
            badge_text="1 file"
        ),
        DetailItem(
            id="root_folder",
            label="All Files",
            render=render_folder_overview,
            data_loader=load_folder_data,
            badge_text="3 items",
            badge_color=badge_colors.success
        )
    ]
)

# Generate router
browser_router = file_browser.create_router(prefix="/files")

# In your FastHTML app, register the router:
# from cjm_fasthtml_app_core.core.routing import register_routes
# register_routes(app, browser_router)
#
# Or directly:
# browser_router.to_app(app)

In [ ]:
# Test master-detail structure
print(f"Interface has {len(file_browser.item_index)} items")
print(f"Default item: {file_browser.default_item}")
print(f"Item IDs: {list(file_browser.item_index.keys())}")
print(f"Master width: {file_browser.master_width}")

Interface has 4 items
Default item: doc1
Item IDs: ['doc1', 'doc2', 'img1', 'root_folder']
Master width: w-64


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()